In [1]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import altair as alt
from requests.utils import quote
import os
from github import Github
from datetime import timedelta

In [2]:
end_date = "2019-07-01"
start_date = "2019-05-01"
renderer = "kaggle"

In [3]:
# Create the renderer we want
alt.renderers.enable(renderer)

RendererRegistry.enable('kaggle')

In [4]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [5]:
gh = Github(os.environ["GITHUB_ACCESS_TOKEN"])

In [6]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# GitHub
def run_github_query(query, github_base="https://api.github.com/search/issues"):
    gh = Github(os.environ["GITHUB_ACCESS_TOKEN"])
    resp = gh.search_issues(query)
    data = []
    for ii in range(0, 10):
        items = resp.get_page(ii)
        if len(items) > 0:
            items_list = [item.raw_data for item in items]
            data.append(pd.DataFrame.from_records(items_list))
        else:
            break
    if len(data) == 0:
        return pd.DataFrame([])
    else:
        return pd.concat(data)
    

# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [7]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [8]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [9]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (98 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (39 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [JupyterHub, Binder workshop](https://discourse.jupyter.org/t/jupyterhub-binder-workshop) (12 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (14 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [Jupyter community workshop: proposal on diversity, inclusion and accessibility](https://discourse.jupyter.org/t/jupyter-community-workshop-proposal-on-diversity-inclusion-and-accessibility) (20 posts)
* [Sunset the GitHub repo jupyter/help?](https://discourse.jupyter.org/t/sunset-the-github-repo-jupyter-help) (17 posts)
* [Reviving the all-Jupyter community meetings](https://discourse.jupyter.org/t/reviving-the-all-jupyter-community-meetings) (17 posts)

## Active topics on Discourse

These are topics with lots of activity in recent history.

In [10]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (98 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (40 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (39 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (31 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [Rename this category to Jupyter Server?](https://discourse.jupyter.org/t/rename-this-category-to-jupyter-server) (21 posts)
* [Potential collaboration on user research?](https://discourse.jupyter.org/t/potential-collaboration-on-user-research) (21 posts)

In [11]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:            category_id  bookmarked     parent_category  \
  11       accessibility           2      Special Topics   
  9       Admin requests           1       Site Feedback   
  0                 Meta           5                Meta   
  1        announcements           1                Meta   
  6              discuss           2          JupyterHub   
  2           JupyterHub           1          JupyterHub   
  8   Enterprise Gateway           1  Enterprise Gateway   
  3       Community Chat           3      Community Chat   
  4               Binder           6              Binder   
  7              discuss           6              Binder   
  5            BinderHub           1              Binder   
  10    mybinder.org ops           1              Binder   
  
                            parent/category  
  11           Special Topics/accessibility  
  9            Site Feedback/Admin requests  
  0                               Meta/Meta  
  1                      Meta/announcements  
  6                      JupyterHub/discuss  
  2                   JupyterHub/JupyterHub  
  8   Enterprise Gateway/Enterprise Gateway  
  3           Community Chat/Community Chat  
  4                           Binder/Binder  
  7                          Binder/discuss  
  5                        Binder/BinderHub  
  10                Binder/mybinder.org ops  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['accessibility', 'Admin requests', 'Meta', 'announcements', 'discuss', 'JupyterHub', 'Enterprise Gateway', 'Community Chat', 'Binder', 'discuss', 'BinderHub', 'mybinder.org ops']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [12]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [Automatic HTTPS with LetsEncrypt in the context of Binder](https://discourse.jupyter.org/t/automatic-https-with-letsencrypt-in-the-context-of-binder) (12 posts)
* [Creating environment.yml for Python 2 via Binder](https://discourse.jupyter.org/t/creating-environment-yml-for-python-2-via-binder) (11 posts)
* [Can I serve notebooks from my own server?](https://discourse.jupyter.org/t/can-i-serve-notebooks-from-my-own-server) (9 posts)
* [Binderhub button - &lsquo;pull from referrer&rsquo;](https://discourse.jupyter.org/t/binderhub-button-pull-from-referrer) (9 posts)
* [Why does nb2kg connect to the kernel via EG rather than directly?](https://discourse.jupyter.org/t/why-does-nb2kg-connect-to-the-kernel-via-eg-rather-than-directly) (6 posts)
* [&ldquo;Astronomy Picture of the Day&rdquo; tutorial problem. Version mismatch?](https://discourse.jupyter.org/t/astronomy-picture-of-the-day-tutorial-problem-version-mismatch) (5 posts)
* [Cells not importing modules anymore](https://discourse.jupyter.org/t/cells-not-importing-modules-anymore) (5 posts)
* [HTTPS with Let&rsquo;s Encrypt on JupyterHub in Kubernetes](https://discourse.jupyter.org/t/https-with-lets-encrypt-on-jupyterhub-in-kubernetes) (4 posts)
* [Jupyter-server-proxy Gallery?](https://discourse.jupyter.org/t/jupyter-server-proxy-gallery) (4 posts)
* [Is there a way for proxying an internal web service in a cell? (specially when in a binderhub)](https://discourse.jupyter.org/t/is-there-a-way-for-proxying-an-internal-web-service-in-a-cell-specially-when-in-a-binderhub) (3 posts)

In [13]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:                          category_id  bookmarked     parent_category  \
  9                                Q&A           2                 Q&A   
  14                          Notebook           1            Notebook   
  13                         nbconvert           0            Notebook   
  2                               Meta           1                Meta   
  8                         JupyterLab           1          JupyterLab   
  12                              team           1          JupyterHub   
  0   Zero to JupyterHub on Kubernetes           0          JupyterHub   
  3                         JupyterHub           0          JupyterHub   
  11                        Governance           1          Governance   
  10                Enterprise Gateway           1  Enterprise Gateway   
  4                     Community Chat           0      Community Chat   
  1                          repo help           1              Binder   
  5                             Binder           1              Binder   
  6                          BinderHub           1              Binder   
  7                            discuss           1              Binder   
  
                                  parent/category  
  9                                       Q&A/Q&A  
  14                            Notebook/Notebook  
  13                           Notebook/nbconvert  
  2                                     Meta/Meta  
  8                         JupyterLab/JupyterLab  
  12                              JupyterHub/team  
  0   JupyterHub/Zero to JupyterHub on Kubernetes  
  3                         JupyterHub/JupyterHub  
  11                        Governance/Governance  
  10        Enterprise Gateway/Enterprise Gateway  
  4                 Community Chat/Community Chat  
  1                              Binder/repo help  
  5                                 Binder/Binder  
  6                              Binder/BinderHub  
  7                                Binder/discuss  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['Q&A', 'Notebook', 'nbconvert', 'Meta', 'JupyterLab', 'team', 'Zero to JupyterHub on Kubernetes', 'JupyterHub', 'Governance', 'Enterprise Gateway', 'Community Chat', 'repo help', 'Binder', 'BinderHub', 'discuss']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## User activity in Discourse

These are the top posters and top "like-ers" in the Discourse

In [14]:
def plot_user_data(users, column):
    plt_data = users.sort_values(column, ascending=False).head(50)
    ch = alt.Chart(data=plt_data).mark_bar().encode(
        x=alt.X("username", sort=plt_data['username'].tolist()),
        y=column
    )
    return ch

In [15]:
url = "https://discourse.jupyter.org/directory_items.json"
params = {"period": "quarterly", "order": "post_count"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
users = pd.DataFrame(resp.json()['directory_items'])
users['username'] = users['user'].map(lambda a: a['username'])

In [16]:
plot_user_data(users.head(50), 'post_count')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             78    6          209             132         172        1324   
  1             90    3          195             115         133         968   
  2             69   24           18              53          64         659   
  3             66   12            9              49          50         868   
  4             80    8            6              48          44         745   
  5             69  116           21              47          39         586   
  6             77   73           13              30          31        1500   
  7             14  220            1              24          27         186   
  8             64  239            6              11          26         477   
  9             81   30           78              18          24         312   
  10            60   99           45              19          22         535   
  11            19  234            5               3          16          84   
  15            29   60           12               9          14         336   
  14            30  372            6              13          14         208   
  13            28  419            0               3          14          77   
  12            26  485           36              14          14         227   
  16            11  254           13               1          12         175   
  17            62   11           56               8          12         735   
  18            17  145            0               9          10         172   
  19            34  432           28               6          10         272   
  20            32  217            9               6           9         181   
  21            23  333           13               7           9          22   
  22            10  510            0               6           8          10   
  23             8  390            1               1           8          51   
  24            45  222            4              11           8         128   
  28             9  566            0               1           7          29   
  30            86   77            6               6           7         830   
  29            18  227           10               2           7         272   
  25            14  487            0               0           7          83   
  27            14  351            0               8           7          60   
  26            17  462            0               1           7          73   
  31            13    9            0               7           6          92   
  32             9  195            0               1           6          33   
  33            17    4            0              17           6          42   
  34             3  679            0               8           6          55   
  35            22  281            4               8           6          41   
  36            42  303            5               3           6         163   
  37            19  237            1              26           6          52   
  38            24  134            8              17           6         215   
  44             5  565            2               3           5          33   
  48            31  184           14               2           5         260   
  47            24    7           21               5           5         223   
  46            45  298            0               4           5         294   
  45            41   67            6               7           5         307   
  39             6  562            0               0           5          22   
  43            76  229            6               0           5         141   
  42             6  556            3               6           5          65   
  41            11  430            2               3           5          41   
  40            11  247            6              14           5  

**Discourse users sorted by likes given**

In [17]:
plot_user_data(users.head(50), 'likes_given')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             78    6          209             132         172        1324   
  1             90    3          195             115         133         968   
  9             81   30           78              18          24         312   
  17            62   11           56               8          12         735   
  10            60   99           45              19          22         535   
  12            26  485           36              14          14         227   
  19            34  432           28               6          10         272   
  5             69  116           21              47          39         586   
  47            24    7           21               5           5         223   
  2             69   24           18              53          64         659   
  48            31  184           14               2           5         260   
  16            11  254           13               1          12         175   
  6             77   73           13              30          31        1500   
  21            23  333           13               7           9          22   
  15            29   60           12               9          14         336   
  29            18  227           10               2           7         272   
  3             66   12            9              49          50         868   
  20            32  217            9               6           9         181   
  38            24  134            8              17           6         215   
  30            86   77            6               6           7         830   
  43            76  229            6               0           5         141   
  40            11  247            6              14           5         183   
  4             80    8            6              48          44         745   
  45            41   67            6               7           5         307   
  14            30  372            6              13          14         208   
  8             64  239            6              11          26         477   
  11            19  234            5               3          16          84   
  36            42  303            5               3           6         163   
  24            45  222            4              11           8         128   
  35            22  281            4               8           6          41   
  42             6  556            3               6           5          65   
  44             5  565            2               3           5          33   
  41            11  430            2               3           5          41   
  37            19  237            1              26           6          52   
  49             8  455            1               0           4         228   
  23             8  390            1               1           8          51   
  7             14  220            1              24          27         186   
  34             3  679            0               8           6          55   
  33            17    4            0              17           6          42   
  32             9  195            0               1           6          33   
  31            13    9            0               7           6          92   
  39             6  562            0               0           5          22   
  28             9  566            0               1           7          29   
  27            14  351            0               8           7          60   
  26            17  462            0               1           7          73   
  22            10  510            0               6           8          10   
  18            17  145            0               9          10         172   
  46            45  298            0               4           5         294   
  13            28  419            0               3          14  

**Discourse users sorted by likes given**

In [18]:
plot_user_data(users.head(50), 'likes_received')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             78    6          209             132         172        1324   
  1             90    3          195             115         133         968   
  2             69   24           18              53          64         659   
  3             66   12            9              49          50         868   
  4             80    8            6              48          44         745   
  5             69  116           21              47          39         586   
  6             77   73           13              30          31        1500   
  37            19  237            1              26           6          52   
  7             14  220            1              24          27         186   
  10            60   99           45              19          22         535   
  9             81   30           78              18          24         312   
  38            24  134            8              17           6         215   
  33            17    4            0              17           6          42   
  12            26  485           36              14          14         227   
  40            11  247            6              14           5         183   
  14            30  372            6              13          14         208   
  8             64  239            6              11          26         477   
  24            45  222            4              11           8         128   
  15            29   60           12               9          14         336   
  18            17  145            0               9          10         172   
  17            62   11           56               8          12         735   
  35            22  281            4               8           6          41   
  34             3  679            0               8           6          55   
  27            14  351            0               8           7          60   
  31            13    9            0               7           6          92   
  45            41   67            6               7           5         307   
  21            23  333           13               7           9          22   
  30            86   77            6               6           7         830   
  22            10  510            0               6           8          10   
  20            32  217            9               6           9         181   
  19            34  432           28               6          10         272   
  42             6  556            3               6           5          65   
  47            24    7           21               5           5         223   
  46            45  298            0               4           5         294   
  36            42  303            5               3           6         163   
  41            11  430            2               3           5          41   
  44             5  565            2               3           5          33   
  13            28  419            0               3          14          77   
  11            19  234            5               3          16          84   
  29            18  227           10               2           7         272   
  48            31  184           14               2           5         260   
  26            17  462            0               1           7          73   
  16            11  254           13               1          12         175   
  32             9  195            0               1           6          33   
  28             9  566            0               1           7          29   
  23             8  390            1               1           8          51   
  25            14  487            0               0           7          83   
  43            76  229            6               0           5         141   
  39             6  562            0               0           5  

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [19]:
from datetime import timedelta

orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples"]
orgs_md = []
for org in orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)

In [20]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-05-01** to **2019-07-01**

In [21]:
# Define our query object that we'll re-use for github search
class GitHubGraphQlQuery():
    def __init__(self, query):
        self.query = query
        self.headers = {"Authorization": "Bearer %s" % os.environ['GITHUB_ACCESS_TOKEN']}
        self.gql_template = """
{
  search(%s) {
    issueCount
    pageInfo {
        endCursor
        hasNextPage
    }
    nodes {
      ... on PullRequest {
        state
        id
        title
        url
        createdAt
        updatedAt
        number
        author {
          login
          url
        }
        mergedBy {
          login
          url
        }
        comments(last: 15) {
          edges {
            node {
              authorAssociation
              createdAt
              lastEditedAt
              url
              repository {
                id
              }
              author {
                login
                url
              }
            }
          }
        }
      }
      ... on Issue {
        url
        title
        state
        createdAt
        updatedAt
        number
        author {
          login
          url
        }
        comments(last: 15) {
          edges {
            node {
              author {
                login
                url
              }
              authorAssociation
              createdAt
              lastEditedAt
              repository {
                id
              }
              url
            }
          }
        }
      }
    }
  }
}
"""
    def request(self, n_pages=100, n_per_page=50):
        self.raw_data = []
        for ii in range(n_pages):
            search_query = ["first: %s" % n_per_page, 'query: "%s"' % self.query, 'type: ISSUE']
            if ii != 0:
                search_query.append('after: "%s"' % pageInfo['endCursor'])

            this_query = self.gql_template % ', '.join(search_query)
            request = requests.post('https://api.github.com/graphql', json={'query': this_query}, headers=self.headers)
            if request.status_code != 200:
                raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, this_query))
            if "errors" in request.json().keys():
                raise Exception("Query failed to run with error {}. {}".format(request.json()['errors'], this_query))
            self.request = request

            # Parse the response
            json = request.json()['data']['search']
            if ii == 0:
                print("Found {} items, which will take {} pages".format(json['issueCount'], int(np.ceil(json['issueCount'] / n_per_page))))
            self.raw_data.append(json)
            pageInfo = json['pageInfo']
            self.last_query = this_query
            if pageInfo['hasNextPage'] is False:
                break
        
        if self.raw_data[0]['issueCount'] == 0:
            print("Found no entries for query {}".format(self.query))
            self.data = None
            return
        
        # Add some extra fields
        self.data = pd.DataFrame([jj for ii in self.raw_data for jj in ii['nodes']])
        self.data['username'] = self.data['author'].map(lambda a: a['login'])
        self.data['username_url'] = self.data['author'].map(lambda a: a['url'])
        self.data['org'] = self.data['url'].map(lambda a: a.split('/')[3])
        self.data['repo'] = self.data['url'].map(lambda a: a.split('/')[4])

In [22]:
responses = []
for org in orgs:
    query_prs = f"is:merged user:{org} archived:false closed:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
merged = pd.concat([ii.data for ii in responses])

Found 238 items, which will take 5 pages
Found 248 items, which will take 5 pages
Found 211 items, which will take 5 pages
Found 79 items, which will take 2 pages
Found 53 items, which will take 2 pages
Found 3 items, which will take 1 pages


In [23]:
# Add a PR-specific field for closed PRs
merged['closed_by'] = merged['mergedBy'].map(lambda a: a['login'])
merged['closed_by_url'] = merged['mergedBy'].map(lambda a: a['url'])

In [24]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)

## Commentors

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [25]:
alt.Chart(data=prs_by_repo, title="Merged PRs in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                         repo  author
  57       jupyterlab                   jupyterlab     173
  61       jupyterlab               jupyterlab-git       9
  68       jupyterlab            jupyterlab_server       9
  56       jupyterlab            jupyter-renderers       4
  59       jupyterlab     jupyterlab-data-explorer       3
  60       jupyterlab              jupyterlab-demo       3
  58       jupyterlab        jupyterlab-commenting       2
  66       jupyterlab               jupyterlab-toc       2
  55       jupyterlab    extension-cookiecutter-js       1
  62       jupyterlab            jupyterlab-github       1
  63       jupyterlab             jupyterlab-latex       1
  64       jupyterlab  jupyterlab-metadata-service       1
  65       jupyterlab         jupyterlab-telemetry       1
  67       jupyterlab          jupyterlab_pygments       1
  44       jupyterhub          mybinder.org-deploy      86
  50       jupyterhub      the-littlest-jupyterhub      39
  36       jupyterhub                    binderhub      21
  41       jupyterhub                   jupyterhub      19
  54       jupyterhub       zero-to-jupyterhub-k8s      18
  33       jupyterhub         alabaster-jupyterhub       9
  34       jupyterhub                 batchspawner       7
  45       jupyterhub                  nbgitpuller       7
  49       jupyterhub                 team-compass       7
  46       jupyterhub               oauthenticator       5
  37       jupyterhub                   chartpress       3
  40       jupyterhub         jupyter-server-proxy       3
  35       jupyterhub                       binder       2
  38       jupyterhub      configurable-http-proxy       2
  52       jupyterhub                traefik-proxy       2
  53       jupyterhub                  wrapspawner       2
  ..              ...                          ...     ...
  29  jupyter-widgets                    pythreejs       4
  30  jupyter-widgets                   traittypes       1
  32  jupyter-widgets          widget-cookiecutter       1
  21          jupyter                     nbgrader      49
  23          jupyter                     notebook      40
  25          jupyter                  repo2docker      30
  19          jupyter                    nbconvert      25
  10          jupyter           enterprise_gateway      24
  12          jupyter                 jupyter-book      12
  9           jupyter                docker-stacks      11
  13          jupyter               jupyter-sphinx       9
  14          jupyter               jupyter_client       9
  15          jupyter                 jupyter_core       9
  24          jupyter                    qtconsole       8
  16          jupyter               jupyter_server       5
  26          jupyter                    telemetry       4
  17          jupyter               kernel_gateway       3
  18          jupyter                        nb2kg       3
  22          jupyter                     nbviewer       3
  11          jupyter                      jupyter       2
  20          jupyter                       nbdime       2
  5           ipython                      ipython      42
  3           ipython                    ipykernel       4
  8           ipython                    traitlets       3
  4           ipython                  ipyparallel       2
  6           ipython             ipython-in-depth       1
  7           ipython              ipython-website       1
  0   binder-examples           getting-data-xroot       1
  1   binder-examples                        latex       1
  2   binder-examples             python-conda_pip       1
  
  [69 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['jupyterlab', 'jupyterlab-git', 'jupyterlab_server', 'jupyter-renderers', 'jupyterlab-data-explorer', 'jupyterlab-demo', 'jupyterlab-commenting', 'jupyterla

### A list of merged PRs by project

In [26]:
from ipywidgets import widgets, HTML
from ipywidgets import HTML
from markdown import markdown 

In [27]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['username']
            user_url = pr['username_url']
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['closed_by']
            pr_closedby_url = pr['closed_by_url']
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

Authoring and merging stats by repo

In [28]:
merged_by = pd.Series(merged_by, name='merger')
pr_by = pd.Series(pr_by, name='author')
stats = pd.merge(merged_by, pr_by, left_index=True, right_index=True)
stats.index.name = "username"
stats.columns.name = "type"
stats = stats.stack()
stats.name = "actions"
stats = stats.reset_index()
ch = alt.Chart(data=stats)
bar = ch.mark_bar().encode(
    x="username",
    y="actions",
    color="type"
)
bar

Chart({
  data:           username    type  actions
  0        choldgraf  merger       46
  1        choldgraf  author       29
  2            minrk  merger       41
  3            minrk  author       19
  4        blink1073  merger       73
  5        blink1073  author       18
  6          Carreau  merger       42
  7          Carreau  author       22
  8          parente  merger       14
  9          parente  author        5
  10     kevin-bates  merger       18
  11     kevin-bates  author       18
  12       akchinSTC  merger        3
  13       akchinSTC  author        4
  14        lresende  merger       14
  15        lresende  author        4
  16        akhmerov  merger        6
  17        akhmerov  author        2
  18   SylvainCorlay  merger       30
  19   SylvainCorlay  author       20
  20        jbweston  merger        2
  21        jbweston  author        7
  22           MSeal  merger       20
  23           MSeal  author        4
  24       takluyver  merger       35
  25       takluyver  author        8
  26          ivanov  merger        3
  27          ivanov  author        4
  28        willingc  merger        2
  29        willingc  author        3
  ..             ...     ...      ...
  52       sgibson91  merger        4
  53       sgibson91  author        5
  54      ryanlovett  merger        2
  55      ryanlovett  author        2
  56          henchc  merger        8
  57          henchc  author        3
  58  GeorgianaElena  merger        5
  59  GeorgianaElena  author        7
  60      ian-r-rose  merger       28
  61      ian-r-rose  author       20
  62         gnestor  merger        4
  63         gnestor  author        4
  64        aschlaep  merger        2
  65        aschlaep  author        6
  66  saulshanabrook  merger       10
  67  saulshanabrook  author       13
  68       ellisonbg  merger        5
  69       ellisonbg  author       10
  70          afshin  merger       11
  71          afshin  author       12
  72      fcollonval  merger        1
  73      fcollonval  author        7
  74          acu192  merger        2
  75          acu192  author        2
  76          hoo761  merger        1
  77          hoo761  author        1
  78           dleen  merger        3
  79           dleen  author        2
  80         Zsailer  merger        1
  81         Zsailer  author        9
  
  [82 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'type',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'actions',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [29]:
responses = []
for org in orgs:
    query_prs = f"is:open is:issue user:{org} archived:false created:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
created = pd.concat([ii.data for ii in responses])
created['n_comments'] = created['comments'].map(lambda a: len(a['edges']))

Found 191 items, which will take 4 pages
Found 196 items, which will take 4 pages
Found 169 items, which will take 4 pages
Found 46 items, which will take 1 pages
Found 45 items, which will take 1 pages
Found 1 items, which will take 1 pages


In [30]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title="Issues created in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                           repo  number  \
  0   binder-examples                   jupyter-rise       1   
  1           ipython                      ipykernel       6   
  2           ipython                    ipyparallel       4   
  3           ipython                        ipython      35   
  4           jupyter                  accessibility       1   
  5           jupyter                     dashboards       1   
  6           jupyter                         design       1   
  7           jupyter                  docker-stacks      13   
  8           jupyter             enterprise_gateway       8   
  9           jupyter                        jupyter       3   
  10          jupyter                   jupyter-book      15   
  11          jupyter                 jupyter-sphinx       2   
  12          jupyter                 jupyter_client       5   
  13          jupyter                   jupyter_core       1   
  14          jupyter            jupyter_kernel_test       1   
  15          jupyter                 jupyter_server       8   
  16          jupyter                 kernel_gateway       1   
  17          jupyter                      nbconvert      12   
  18          jupyter                         nbdime       3   
  19          jupyter                       nbgrader      27   
  20          jupyter                       nbviewer       7   
  21          jupyter                       notebook      64   
  22          jupyter                      qtconsole       1   
  23          jupyter                    repo2docker      20   
  24          jupyter                       steering       1   
  25          jupyter                      terminado       1   
  26  jupyter-widgets                     ipyleaflet      12   
  27  jupyter-widgets                     ipywidgets      24   
  28  jupyter-widgets                      pythreejs       8   
  29  jupyter-widgets                       tutorial       2   
  ..              ...                            ...     ...   
  42       jupyterhub       jupyterhub-deploy-docker       2   
  43       jupyterhub                    kubespawner       4   
  44       jupyterhub              ldapauthenticator       3   
  45       jupyterhub               ltiauthenticator       2   
  46       jupyterhub            mybinder.org-deploy      11   
  47       jupyterhub            nativeauthenticator       1   
  48       jupyterhub                    nbgitpuller       9   
  49       jupyterhub                 oauthenticator       8   
  50       jupyterhub                    sudospawner       1   
  51       jupyterhub                 systemdspawner       2   
  52       jupyterhub                   team-compass      13   
  53       jupyterhub        the-littlest-jupyterhub      21   
  54       jupyterhub                  traefik-proxy       2   
  55       jupyterhub                    wrapspawner       2   
  56       jupyterhub         zero-to-jupyterhub-k8s      29   
  57       jupyterlab      extension-cookiecutter-ts       1   
  58       jupyterlab              jupyter-renderers       1   
  59       jupyterlab                     jupyterlab     127   
  60       jupyterlab            jupyterlab-celltags       3   
  61       jupyterlab          jupyterlab-commenting      10   
  62       jupyterlab       jupyterlab-data-explorer      10   
  63       jupyterlab                 jupyterlab-git       6   
  64       jupyterlab              jupyterlab-github       1   
  65       jupyterlab        jupyterlab-google-drive       1   
  66       jupyterlab    jupyterlab-metadata-service       1   
  67       jupyterlab           jupyterlab-telemetry       2   
  68       jupyterlab                 jupyterlab-toc       1   
  69       jupyterlab                 jupyterlab_app       1   
  70       jupyterlab  scipy2018-jupyterlab-tutorial       1   
  71       jupyterlab                   team-compass       3   
  
               

In [31]:
responses = []
for org in orgs:
    query_prs = f"is:issue user:{org} closed:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
closed = pd.concat([ii.data for ii in responses])
closed['n_comments'] = closed['comments'].map(lambda a: len(a['edges']))

Found 141 items, which will take 3 pages
Found 233 items, which will take 5 pages
Found 216 items, which will take 5 pages
Found 50 items, which will take 1 pages
Found 33 items, which will take 1 pages
Found 1 items, which will take 1 pages


In [32]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title="Issues closed in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0   binder-examples                      bokeh       1   
  1           ipython                  ipykernel       3   
  2           ipython                ipyparallel       1   
  3           ipython                    ipython      28   
  4           ipython                  traitlets       1   
  5           jupyter              atom-notebook       1   
  6           jupyter              docker-stacks       9   
  7           jupyter         enterprise_gateway      24   
  8           jupyter                    jupyter       3   
  9           jupyter               jupyter-book      16   
  10          jupyter             jupyter-sphinx       8   
  11          jupyter             jupyter_client       1   
  12          jupyter               jupyter_core      10   
  13          jupyter             jupyter_server       8   
  14          jupyter             kernel_gateway       3   
  15          jupyter                      nb2kg       2   
  16          jupyter                  nbconvert      25   
  17          jupyter                     nbdime       1   
  18          jupyter                   nbgrader      46   
  19          jupyter                   nbviewer       4   
  20          jupyter                   notebook      47   
  21          jupyter                  qtconsole       5   
  22          jupyter                repo2docker      19   
  23          jupyter                  terminado       1   
  24  jupyter-widgets                 ipyleaflet      18   
  25  jupyter-widgets                 ipywidgets      27   
  26  jupyter-widgets                  pythreejs       3   
  27  jupyter-widgets                   tutorial       2   
  28       jupyterhub       alabaster-jupyterhub       1   
  29       jupyterhub               batchspawner       2   
  30       jupyterhub                  binderhub      11   
  31       jupyterhub    configurable-http-proxy       1   
  32       jupyterhub              dockerspawner       1   
  33       jupyterhub       jupyter-server-proxy       4   
  34       jupyterhub                 jupyterhub      26   
  35       jupyterhub   jupyterhub-deploy-docker       1   
  36       jupyterhub                kubespawner       2   
  37       jupyterhub          ldapauthenticator       1   
  38       jupyterhub        mybinder.org-deploy      10   
  39       jupyterhub                nbgitpuller      10   
  40       jupyterhub             oauthenticator       2   
  41       jupyterhub             systemdspawner       1   
  42       jupyterhub               team-compass      10   
  43       jupyterhub    the-littlest-jupyterhub      45   
  44       jupyterhub           tmpauthenticator       1   
  45       jupyterhub              traefik-proxy       2   
  46       jupyterhub     zero-to-jupyterhub-k8s      10   
  47       jupyterlab  extension-cookiecutter-js       1   
  48       jupyterlab          jupyter-renderers       1   
  49       jupyterlab                 jupyterlab     191   
  50       jupyterlab   jupyterlab-data-explorer       7   
  51       jupyterlab            jupyterlab-demo       2   
  52       jupyterlab             jupyterlab-git      11   
  53       jupyterlab          jupyterlab-github       1   
  54       jupyterlab             jupyterlab-toc       1   
  55       jupyterlab          jupyterlab_server       1   
  
                                  org/repo  
  0                  binder-examples/bokeh  
  1                      ipython/ipykernel  
  2                    ipython/ipyparallel  
  3                        ipython/ipython  
  4                      ipython/traitlets  
  5                  jupyter/atom-notebook  
  6                  jupyter/docker-stacks  
  7             jupyter/enterprise_gateway  
  8                        jupyter/jupyter  
  9                   jupyter/jupyter-book  
  10                jupyter/jupyter-sphinx  
  11                jup

### A list of recent issues

In [33]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(created.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for (org, repo), issues in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, issue in issues.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['username']
            user_url = issue['username_url']
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)
    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last 30 days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 30 days

## Commenters across repositories

In [34]:
comments = []
for items in [created, merged, closed]:
    for ii, irow in items.iterrows():
        icomments = [ii['node'] for ii in irow['comments']['edges']]
        icomments = pd.DataFrame(icomments)
        icomments['repo'] = irow['repo']
        icomments['org'] = irow['org']
        comments.append(icomments)
comments = pd.concat(comments)
comments = comments.query("createdAt > @start_date and createdAt < @end_date")
comments['username'] = comments['author'].map(lambda a: a['login'])
comments['username_url'] = comments['author'].map(lambda a: a['url'])

/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [35]:
commentors = (
    comments
    .groupby(['org', 'repo', 'username'])
    .count()['author']
    .reset_index()
    .sort_values(['org', 'author'], ascending=False)
)

In [36]:
for iorg, idata in commentors.groupby(['org']):
    title = f"Top 50 commentors for {iorg} in the last 30 days"
    ch = alt.Chart(data=idata.head(50), width=1000, title=title).mark_bar().encode(
        x=alt.X('username', sort=alt.Sort(idata['username'].tolist())),
        y='author',
        color='org',
    )
    display(ch)

Chart({
  data:                org                repo             username  author
  0  binder-examples               bokeh            choldgraf       2
  1  binder-examples               bokeh  mathematicalmichael       1
  2  binder-examples  getting-data-xroot            choldgraf       1
  3  binder-examples  getting-data-xroot      matthewfeickert       1
  4  binder-examples        jupyter-rise            choldgraf       1
  5  binder-examples        jupyter-rise  mathematicalmichael       1
  6  binder-examples    python-conda_pip            choldgraf       1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      sort: Sort(['choldgraf', 'mathematicalmichael', 'choldgraf', 'matthewfeickert', 'choldgraf', 'mathematicalmichael', 'choldgraf']),
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 commentors for binder-examples in the last 30 days',
  width: 1000
})

Chart({
  data:         org             repo        username  author
  16  ipython          ipython         Carreau      66
  59  ipython        traitlets        rmorshea       7
  11  ipython        ipykernel         hroncok       6
  13  ipython        ipykernel           minrk       5
  17  ipython          ipython  LucianaMarques       5
  10  ipython        ipykernel      ccordoba12       4
  34  ipython          ipython         efiring       4
  49  ipython          ipython        stonebig       4
  56  ipython        traitlets     kevin-bates       4
  23  ipython          ipython        asmeurer       3
  47  ipython          ipython    sanketShetty       3
  7   ipython        ipykernel         Carreau       2
  9   ipython        ipykernel       blink1073       2
  15  ipython      ipyparallel           minrk       2
  19  ipython          ipython       Pragnya02       2
  21  ipython          ipython  angertdevsingh       2
  25  ipython          ipython         blueyed       2
  30  ipython          ipython        dhomeier       2
  32  ipython          ipython          dlukes       2
  33  ipython          ipython     drunkwcodes       2
  37  ipython          ipython        gpotter2       2
  40  ipython          ipython        jfhbrook       2
  42  ipython          ipython          kmaork       2
  44  ipython          ipython         mlucool       2
  45  ipython          ipython      pierstitus       2
  46  ipython          ipython           rgson       2
  50  ipython          ipython       tacaswell       2
  58  ipython        traitlets          rgbkrk       2
  8   ipython        ipykernel         Chilipp       1
  12  ipython        ipykernel      jamesmyatt       1
  14  ipython        ipykernel          noklam       1
  18  ipython          ipython           MSeal       1
  20  ipython          ipython            alok       1
  22  ipython          ipython         anntzer       1
  24  ipython          ipython        bashtage       1
  26  ipython          ipython      ccordoba12       1
  27  ipython          ipython           cdeil       1
  28  ipython          ipython         csheaff       1
  29  ipython          ipython          daharn       1
  31  ipython          ipython         dickmao       1
  35  ipython          ipython  farisachugthai       1
  36  ipython          ipython       gdhameeja       1
  38  ipython          ipython         hroncok       1
  39  ipython          ipython      ikizhvatov       1
  41  ipython          ipython       jmansilla       1
  43  ipython          ipython           kmtac       1
  48  ipython          ipython         scopatz       1
  51  ipython          ipython       takluyver       1
  52  ipython  ipython-website         Carreau       1
  53  ipython  ipython-website       blink1073       1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      sort: Sort(['Carreau', 'rmorshea', 'hroncok', 'minrk', 'LucianaMarques', 'ccordoba12', 'efiring', 'stonebig', 'kevin-bates', 'asmeurer', 'sanketShetty', 'Carreau', 'blink1073', 'minrk', 'Pragnya02', 'angertdevsingh', 'blueyed', 'dhomeier', 'dlukes', 'drunkwcodes', 'gpotter2', 'jfhbrook', 'kmaork', 'mlucool', 'pierstitus', 'rgson', 'tacaswell', 'rgbkrk', 'Chilipp', 'jamesmyatt', 'noklam', 'MSeal', 'alok', 'anntzer', 'bashtage', 'ccordoba12', 'cdeil', 'csheaff', 'daharn', 'dickmao', 'farisachugthai', 'gdhameeja', 'hroncok', 'ikizhvatov', 'jmansilla', 'kmtac', 'scopatz', 'takluyver', 'Carreau', 'blink1073', 'gpotter2', 'frenzymadness', 'minrk']),
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 commentors for ipython in the last 30 days',
  width: 1000
})

Chart({
  data:          org                repo         username  author
  197  jupyter            nbgrader         jhamrick     135
  292  jupyter         repo2docker          betatim      99
  92   jupyter  enterprise_gateway      kevin-bates      75
  276  jupyter            notebook        takluyver      56
  106  jupyter        jupyter-book        choldgraf      51
  157  jupyter           nbconvert            MSeal      48
  313  jupyter         repo2docker        yuvipanda      37
  296  jupyter         repo2docker        choldgraf      27
  208  jupyter            nbgrader          rkdarst      23
  79   jupyter       docker-stacks          parente      22
  141  jupyter      jupyter_server          Zsailer      22
  297  jupyter         repo2docker     consideRatio      20
  176  jupyter           nbconvert         t-makaro      18
  203  jupyter            nbgrader          nthiery      17
  115  jupyter        jupyter-book      psychemedia      15
  120  jupyter      jupyter-sphinx         akhmerov      14
  88   jupyter  enterprise_gateway         echarles      13
  257  jupyter            notebook      kevin-bates      13
  138  jupyter        jupyter_core        takluyver      12
  96   jupyter  enterprise_gateway         suryag10      11
  170  jupyter           nbconvert  maartenbreddels      10
  273  jupyter            notebook       ssanderson      10
  298  jupyter         repo2docker     davidanthoff      10
  319  jupyter           telemetry       jaipreet-s      10
  94   jupyter  enterprise_gateway        michzimny       9
  147  jupyter      jupyter_server         rolweber       9
  205  jupyter            nbgrader        perllaghu       9
  255  jupyter            notebook    joshuazeltser       9
  288  jupyter         repo2docker       Xarthisius       9
  153  jupyter               nb2kg     Litchilitchy       8
  154  jupyter               nb2kg      kevin-bates       8
  192  jupyter            nbgrader       gertingold       8
  263  jupyter            notebook         lresende       8
  274  jupyter            notebook         stonebig       8
  275  jupyter            notebook         t-makaro       8
  281  jupyter           qtconsole       ccordoba12       8
  318  jupyter           telemetry          Zsailer       8
  121  jupyter      jupyter-sphinx         jbweston       7
  185  jupyter            nbgrader          Sefriol       7
  191  jupyter            nbgrader          dsblank       7
  206  jupyter            nbgrader      psychemedia       7
  305  jupyter         repo2docker            minrk       7
  133  jupyter      jupyter_client           rgbkrk       6
  140  jupyter      jupyter_server    SylvainCorlay       6
  182  jupyter              nbdime          vidartf       6
  220  jupyter            nbviewer          parente       6
  321  jupyter           telemetry        yuvipanda       6
  66   jupyter       docker-stacks          betatim       5
  84   jupyter  enterprise_gateway         IMAM9AIS       5
  93   jupyter  enterprise_gateway         lresende       5,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      sort: Sort(['jhamrick', 'betatim', 'kevin-bates', 'takluyver', 'choldgraf', 'MSeal', 'yuvipanda', 'choldgraf', 'rkdarst', 'parente', 'Zsailer', 'consideRatio', 't-makaro', 'nthiery', 'psychemedia', 'akhmerov', 'echarles', 'kevin-bates', 'takluyver', 'suryag10', 'maartenbreddels', 'ssanderson', 'davidanthoff', 'jaipreet-s', 'michzimny', 'rolweber', 'perllaghu', 'joshuazeltser', 'Xarthisius', 'Litchilitchy', 'kevin-bates', 'gertingold', 'lresende', 'stonebig', 't-makaro', 'ccordoba12', 'Zsailer', 'jbweston', 'Sefriol', 'dsblank', 'psychemedia', 'minrk', 'rgbkrk', 'SylvainCorlay', 'vidartf', 'parente', 'yuvipanda', 'betatim', 'IMAM9AIS', 'lresende', 'cpjobling', 'blink1073', 'echarles', 'bollwyvl', 'MSeal', 'Pierek', 'jasongrout', 'cboettig', 'jlperla', 'consideRatio', 'Arinda

Chart({
  data:                  org        repo         username  author
  334  jupyter-widgets  ipyleaflet      martinRenou      55
  354  jupyter-widgets  ipywidgets       jasongrout      53
  344  jupyter-widgets  ipywidgets    SylvainCorlay      49
  373  jupyter-widgets   pythreejs          vidartf      24
  332  jupyter-widgets  ipyleaflet     fitoprincipe      16
  325  jupyter-widgets  ipyleaflet        Kirill888      13
  364  jupyter-widgets  ipywidgets          vidartf      13
  380  jupyter-widgets    tutorial          mwcraig      12
  353  jupyter-widgets  ipywidgets            hainm      11
  356  jupyter-widgets  ipywidgets  maartenbreddels       9
  345  jupyter-widgets  ipywidgets           aa2858       7
  333  jupyter-widgets  ipyleaflet        kgiacobbi       6
  357  jupyter-widgets  ipywidgets          mwcraig       6
  343  jupyter-widgets  ipywidgets        Ricyteach       5
  367  jupyter-widgets   pythreejs         aliddell       5
  329  jupyter-widgets  ipyleaflet         deeplook       4
  355  jupyter-widgets  ipywidgets          kafonek       4
  370  jupyter-widgets   pythreejs        jhmitchel       4
  361  jupyter-widgets  ipywidgets       ssanderson       3
  363  jupyter-widgets  ipywidgets          trllrgn       3
  378  jupyter-widgets    tutorial  maartenbreddels       3
  327  jupyter-widgets  ipyleaflet     VasavanThiru       2
  328  jupyter-widgets  ipyleaflet         chbrandt       2
  331  jupyter-widgets  ipyleaflet      dwr-psandhu       2
  337  jupyter-widgets  ipywidgets  DanielAristidou       2
  342  jupyter-widgets  ipywidgets        Liwellyen       2
  347  jupyter-widgets  ipywidgets      andymcarter       2
  348  jupyter-widgets  ipywidgets        beasteers       2
  352  jupyter-widgets  ipywidgets        choldgraf       2
  360  jupyter-widgets  ipywidgets       rskabelund       2
  368  jupyter-widgets   pythreejs           ataiya       2
  371  jupyter-widgets   pythreejs           skoch9       2
  381  jupyter-widgets    tutorial          vidartf       2
  324  jupyter-widgets  ipyleaflet        Juanlu001       1
  326  jupyter-widgets  ipyleaflet    SylvainCorlay       1
  330  jupyter-widgets  ipyleaflet          drwelby       1
  335  jupyter-widgets  ipyleaflet         mpdwulit       1
  336  jupyter-widgets  ipywidgets       DYSpider13       1
  338  jupyter-widgets  ipywidgets         DevFabrx       1
  339  jupyter-widgets  ipywidgets   IlyaKazakevich       1
  340  jupyter-widgets  ipywidgets        JohnIssue       1
  341  jupyter-widgets  ipywidgets        Juanlu001       1
  346  jupyter-widgets  ipywidgets         akhmerov       1
  349  jupyter-widgets  ipywidgets            blois       1
  350  jupyter-widgets  ipywidgets           bybyte       1
  351  jupyter-widgets  ipywidgets        chimaobiv       1
  358  jupyter-widgets  ipywidgets       oscar6echo       1
  359  jupyter-widgets  ipywidgets           piyueh       1
  362  jupyter-widgets  ipywidgets    throwable-one       1
  365  jupyter-widgets  ipywidgets           zwelz3       1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      sort: Sort(['martinRenou', 'jasongrout', 'SylvainCorlay', 'vidartf', 'fitoprincipe', 'Kirill888', 'vidartf', 'mwcraig', 'hainm', 'maartenbreddels', 'aa2858', 'kgiacobbi', 'mwcraig', 'Ricyteach', 'aliddell', 'deeplook', 'kafonek', 'jhmitchel', 'ssanderson', 'trllrgn', 'maartenbreddels', 'VasavanThiru', 'chbrandt', 'dwr-psandhu', 'DanielAristidou', 'Liwellyen', 'andymcarter', 'beasteers', 'choldgraf', 'rskabelund', 'ataiya', 'skoch9', 'vidartf', 'Juanlu001', 'SylvainCorlay', 'drwelby', 'mpdwulit', 'DYSpider13', 'DevFabrx', 'IlyaKazakevich', 'JohnIssue', 'Juanlu001', 'akhmerov', 'blois', 'bybyte', 'chimaobiv', 'oscar6echo', 'piyueh', 'throwable-one', 'zwelz3', 'TakodaS', 'jasongrout', 't-suzuki', 'Jamim', 'SylvainCorlay', 'vidartf', 'kohr-h', 'martinRenou', 'wolfv']),
      ty

Chart({
  data:             org                     repo             username  author
  580  jupyterhub  the-littlest-jupyterhub            yuvipanda     120
  501  jupyterhub      mybinder.org-deploy              betatim      63
  392  jupyterhub                binderhub              betatim      49
  540  jupyterhub             team-compass            choldgraf      46
  597  jupyterhub   zero-to-jupyterhub-k8s         consideRatio      41
  539  jupyterhub             team-compass              betatim      40
  522  jupyterhub              nbgitpuller            yuvipanda      35
  479  jupyterhub               jupyterhub              ryogesh      32
  510  jupyterhub      mybinder.org-deploy          mael-le-gal      26
  405  jupyterhub                binderhub            sgibson91      19
  470  jupyterhub               jupyterhub                minrk      18
  503  jupyterhub      mybinder.org-deploy            choldgraf      17
  388  jupyterhub             batchspawner              rkdarst      15
  394  jupyterhub                binderhub            choldgraf      12
  516  jupyterhub              nbgitpuller            choldgraf      12
  553  jupyterhub  the-littlest-jupyterhub       GeorgianaElena      12
  604  jupyterhub   zero-to-jupyterhub-k8s               manics      11
  395  jupyterhub                binderhub         consideRatio      10
  519  jupyterhub              nbgitpuller          parmentelat      10
  549  jupyterhub             team-compass            yuvipanda      10
  483  jupyterhub               jupyterhub            vilhelmen       9
  538  jupyterhub             team-compass              Zsailer       9
  402  jupyterhub                binderhub                minrk       7
  437  jupyterhub               jupyterhub      RandyBetancourt       7
  452  jupyterhub               jupyterhub         consideRatio       7
  505  jupyterhub      mybinder.org-deploy               henchc       7
  514  jupyterhub      mybinder.org-deploy            yuvipanda       7
  530  jupyterhub           oauthenticator                minrk       7
  605  jupyterhub   zero-to-jupyterhub-k8s                minrk       7
  386  jupyterhub             batchspawner           mbmilligan       6
  551  jupyterhub  the-littlest-jupyterhub          CyborgDroid       6
  585  jupyterhub            traefik-proxy                minrk       6
  401  jupyterhub                binderhub             memeplex       5
  414  jupyterhub            dockerspawner            alvinhuff       5
  422  jupyterhub     jupyter-server-proxy           ian-r-rose       5
  431  jupyterhub               jupyterhub       GeorgianaElena       5
  450  jupyterhub               jupyterhub          brayanrodpr       5
  489  jupyterhub              kubespawner         consideRatio       5
  491  jupyterhub              kubespawner               gsemet       5
  545  jupyterhub             team-compass          mael-le-gal       5
  546  jupyterhub             team-compass                minrk       5
  397  jupyterhub                binderhub              jhamman       4
  407  jupyterhub                binderhub            yuvipanda       4
  426  jupyterhub     jupyter-server-proxy             quasiben       4
  458  jupyterhub               jupyterhub              eylenth       4
  462  jupyterhub               jupyterhub             krinsman       4
  468  jupyterhub               jupyterhub  mathematicalmichael       4
  492  jupyterhub              kubespawner       julienchastang       4
  529  jupyterhub           oauthenticator            kpfleming       4
  548  jupyterhub             team-compass             willingc       4,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      sort: Sort(['yuvipanda', 'betatim', 'betatim', 'choldgraf', 'consideRatio', 'betatim', 'yuvipanda', 'ryogesh', 'mael-le-gal', 'sgibson91', 'minrk', 'choldgraf', 'rkdarst', 

Chart({
  data:             org                      repo             username  author
  708  jupyterlab                jupyterlab           jasongrout     479
  664  jupyterlab                jupyterlab            blink1073     239
  711  jupyterlab                jupyterlab  jupyterlab-dev-mode     150
  703  jupyterlab                jupyterlab           ian-r-rose     145
  755  jupyterlab                jupyterlab              vidartf     114
  741  jupyterlab                jupyterlab       saulshanabrook      51
  689  jupyterlab                jupyterlab            ellisonbg      44
  748  jupyterlab                jupyterlab           telamonian      41
  650  jupyterlab                jupyterlab               afshin      39
  749  jupyterlab                jupyterlab            tgeorgeux      20
  775  jupyterlab            jupyterlab-git           jaipreet-s      18
  657  jupyterlab                jupyterlab             aschlaep      17
  683  jupyterlab                jupyterlab             domoritz      16
  753  jupyterlab                jupyterlab              tslaton      16
  707  jupyterlab                jupyterlab              jakevdp      15
  630  jupyterlab                jupyterlab               BoPeng      14
  647  jupyterlab                jupyterlab              Zsailer      13
  765  jupyterlab  jupyterlab-data-explorer       saulshanabrook      13
  698  jupyterlab                jupyterlab              gnestor      11
  710  jupyterlab                jupyterlab                jtpio      11
  686  jupyterlab                jupyterlab         duarteocarmo      10
  665  jupyterlab                jupyterlab             bollwyvl       9
  643  jupyterlab                jupyterlab        SylvainCorlay       8
  704  jupyterlab                jupyterlab              ibdafna       8
  715  jupyterlab                jupyterlab           kdeleeuw11       8
  780  jupyterlab            jupyterlab-git       saulshanabrook       8
  690  jupyterlab                jupyterlab           fcollonval       7
  746  jupyterlab                jupyterlab             stonebig       7
  788  jupyterlab         jupyterlab_server            blink1073       7
  662  jupyterlab                jupyterlab            bionicles       6
  759  jupyterlab                jupyterlab            yuvipanda       6
  768  jupyterlab           jupyterlab-demo           ian-r-rose       6
  776  jupyterlab            jupyterlab-git  mathematicalmichael       6
  624  jupyterlab         jupyter-renderers              gnestor       5
  679  jupyterlab                jupyterlab         davidanthoff       5
  687  jupyterlab                jupyterlab             echarles       5
  705  jupyterlab                jupyterlab               ivanov       5
  706  jupyterlab                jupyterlab           jaipreet-s       5
  623  jupyterlab         jupyter-renderers             domoritz       4
  658  jupyterlab                jupyterlab            athornton       4
  663  jupyterlab                jupyterlab              bjtho08       4
  725  jupyterlab                jupyterlab           michaelaye       4
  735  jupyterlab                jupyterlab               ppeeou       4
  736  jupyterlab                jupyterlab           rahulpshah       4
  750  jupyterlab                jupyterlab             timhoffm       4
  772  jupyterlab            jupyterlab-git         aquirdTurtle       4
  625  jupyterlab         jupyter-renderers           ian-r-rose       3
  633  jupyterlab                jupyterlab           DevinRouth       3
  636  jupyterlab                jupyterlab           IgorDouven       3
  656  jupyterlab                jupyterlab        antonio-rojas       3,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      sort: Sort(['jasongrout', 'blink1073', 'jupyterlab-dev-mode', 'ian-r-rose', 'vidartf', 'saulshanabrook', 'ellisonbg', 'te

In [37]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>